In [5]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras import applications
import warnings
warnings.filterwarnings("ignore")

In [13]:
#limpiar sesiones de keras, para tener la session limpia
K.clear_session()

#cargo datos de entrenamiento y valdidacion
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

#Parametros

#Numero de veces que iteramos en todo el set de datos
epocas=20

#Tamaño de las imagenes estandar
longitud, altura = 150, 150

#Numero de imagenes que se envian en cada paso
batch_size = 32

#Numero de veces que se procesa la infomacion en cada Epoca
pasos = 1000

#Al final de cada Epoca se corren 300 pasos de validacion
validation_steps = 300

#Tamaño despues de cada convulucion
filtrosConv1 = 32
filtrosConv2 = 64


tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 3

#Learning rate
lr = 0.0004

In [14]:
##Pre procesamiento nuestras imagenes
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,  #Rescala las imagenes los pixels van a ir a valores de 0-1
    shear_range=0.2, #inclina imagenes
    zoom_range=0.2, #algunas fotos se les hace zoom a otroas no
    horizontal_flip=True) #invierte imagenes  para aprender direccion

test_datagen = ImageDataGenerator(rescale=1. / 255)

#preprocesa la carpeta de data
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

#pre procesa la carpeta de validacion
validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 999 images belonging to 3 classes.
Found 2043 images belonging to 3 classes.


In [15]:
#Red neuronal sequencial, varias capas aplicadas
cnn = Sequential()
#Primera capa de red neuronal
cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

#Segunda capa de red neuronal
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))


cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

In [16]:
#Optimizar modelo de Redes neuronales
cnn.compile(loss='categorical_crossentropy',
            optimizer=optimizers.Adam(lr=lr),
            metrics=['accuracy'])

#Entrenar nuestra red Neuronal con las imagenes de entrenamiento
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

target_dir = './modelo/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
cnn.save('./modelo/modelo.h5')
cnn.save_weights('./modelo/pesos.h5')

Epoch 1/20
32/32 [==============================] - 58s 2s/step - loss: 1.4414 - acc: 0.5275 - val_loss: 0.7812 - val_acc: 0.5046
Epoch 2/20
32/32 [==============================] - 58s 2s/step - loss: 0.8070 - acc: 0.5596 - val_loss: 0.7039 - val_acc: 0.6055
Epoch 3/20
32/32 [==============================] - 56s 2s/step - loss: 0.7446 - acc: 0.5946 - val_loss: 0.7520 - val_acc: 0.5370
Epoch 4/20
32/32 [==============================] - 56s 2s/step - loss: 0.7179 - acc: 0.6006 - val_loss: 0.7453 - val_acc: 0.5423
Epoch 5/20
32/32 [==============================] - 59s 2s/step - loss: 0.7107 - acc: 0.6216 - val_loss: 0.7250 - val_acc: 0.6021
Epoch 6/20
32/32 [==============================] - 55s 2s/step - loss: 0.7075 - acc: 0.6206 - val_loss: 0.7225 - val_acc: 0.5825
Epoch 7/20
32/32 [==============================] - 52s 2s/step - loss: 0.6780 - acc: 0.6266 - val_loss: 0.6837 - val_acc: 0.6187
Epoch 8/20
32/32 [==============================] - 57s 2s/step - loss: 0.6766 - acc: 0.65

In [25]:
import numpy as np
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

def predict(file):
    x = load_img(file, target_size=(longitud, altura))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = cnn.predict(x)
    result = array[0]
    answer = np.argmax(result)
    if answer == 0:
        print("pred: Perro")
    elif answer == 1:
        print("pred: Gato")
    elif answer == 2:
        print("pred: Gorila")

    return None

In [27]:
predict('dog.4006.jpg')

pred: Perro
